In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [10]:
baseline = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\baseline_data.xlsx")

In [12]:
baseline['race_datetime'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 100 entries, 0 to 99
Series name: race_datetime
Non-Null Count  Dtype         
--------------  -----         
86 non-null     datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 932.0 bytes


In [26]:
baseline

,Unnamed: 0,PCS race entry,Race Name,Year,Winner,GC speciality points,time_trial speciality points,sprint speciality points,climber speciality points,hills speciality points,...,Won How,date,profile_icon,profile_score,departure,arrival,average_temperature,start_time,race_datetime,race_coords
0,0,2017amstelgoldrace,Amstel Gold Race,2017,GILBERT Philippe,4215.0,1426.0,1094.0,3215.0,5140.0,...,Sprint à deux,2017-04-16,"Hills, flat finish",92.0,Maastricht,Berg en Terblijt,NaN,1900-01-01 11:26:00,2017-04-16 11:26:00,"[(50.87808, 5.71738), (50.87819, 5.71747), (50..."
1,1,2018amstelgoldrace,Amstel Gold Race,2018,VALGREN Michael,1188.0,453.0,84.0,955.0,1858.0,...,Sprint à deux,2018-04-15,"Hills, flat finish",103.0,Maastricht,Berg en Terblijt,NaN,1900-01-01 10:35:00,2018-04-15 10:35:00,"[(50.87808, 5.71738), (50.87819, 5.71747), (50..."
2,2,2019amstelgoldrace,Amstel Gold Race,2019,VAN DER POEL Mathieu,1119.0,968.0,1286.0,1042.0,3090.0,...,Sprint of small group,2019-04-21,"Hills, flat finish",108.0,Maastricht,Berg en Terblijt,NaN,1900-01-01 12:35:00,2019-04-21 12:35:00,"[(50.86287, 5.70027), (50.86307, 5.70027), (50..."
3,3,2021amstelgoldrace,Amstel Gold Race,2021,VAN AERT Wout,1171.0,2944.0,1886.0,2044.0,4221.0,...,Sprint of small group,2021-04-18,"Hills, flat finish",81.0,Valkenburg,Berg en Terblijt,NaN,1900-01-01 12:11:00,2021-04-18 12:11:00,"[(50.86699, 5.78322), (50.86697, 5.78322), (50..."
4,4,2022amstelgoldrace,Amstel Gold Race,2022,KWIATKOWSKI Michał,3670.0,3561.0,755.0,3304.0,3579.0,...,Sprint à deux,2022-04-10,"Hills, flat finish",112.0,Maastricht,Valkenburg,10.0,1900-01-01 10:26:00,2022-04-10 10:26:00,"[(50.86307, 5.70026), (50.86312, 5.70026), (50..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3,2019kuurne-brussel-kuurne,Kuurne-Brussel-Kuurne,2019,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
96,3,2020kuurne-brussel-kuurne,Kuurne-Brussel-Kuurne,2020,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
97,3,2021kuurne-brussel-kuurne,Kuurne-Brussel-Kuurne,2021,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
98,3,2022kuurne-brussel-kuurne,Kuurne-Brussel-Kuurne,2022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN


In [20]:
race_name_dict = baseline['Race Name'].to_dict()

In [22]:
race_name_dict

{0: 'Amstel Gold Race',
 1: 'Amstel Gold Race',
 2: 'Amstel Gold Race',
 3: 'Amstel Gold Race',
 4: 'Amstel Gold Race',
 5: 'Amstel Gold Race',
 6: 'De Brabantse Pijl',
 7: 'De Brabantse Pijl',
 8: 'De Brabantse Pijl',
 9: 'De Brabantse Pijl',
 10: 'De Brabantse Pijl',
 11: 'De Brabantse Pijl',
 12: 'De Brabantse Pijl',
 13: 'Dwars door Vlaanderen',
 14: 'Dwars door Vlaanderen',
 15: 'Dwars door Vlaanderen',
 16: 'Dwars door Vlaanderen',
 17: 'Dwars door Vlaanderen',
 18: 'Dwars door Vlaanderen',
 19: 'E3 Saxo Classic',
 20: 'E3 Saxo Classic',
 21: 'E3 Saxo Classic',
 22: 'E3 Saxo Classic',
 23: 'Gent-Wevelgem',
 24: 'Gent-Wevelgem',
 25: 'Gent-Wevelgem',
 26: 'Gent-Wevelgem',
 27: 'Gent-Wevelgem',
 28: 'Gent-Wevelgem',
 29: 'Gent-Wevelgem',
 30: 'La Flèche Wallonne',
 31: 'La Flèche Wallonne',
 32: 'La Flèche Wallonne',
 33: 'La Flèche Wallonne',
 34: 'La Flèche Wallonne',
 35: 'La Flèche Wallonne',
 36: 'La Flèche Wallonne',
 37: 'Liege - Bastogne - Liege',
 38: 'Liege - Bastogne - L

In [24]:
race_name_mapping = dict(zip(baseline["PCS race entry"], baseline["Race Name"]))

In [28]:
import os
import gpxpy

gpx_folder = r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\data\euro classics gpx"

def extract_gpx_coordinates(gpx_file, num_samples=10):
    with open(gpx_file, "r") as f:
        gpx = gpxpy.parse(f)
    
    coordinates = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                coordinates.append((point.latitude, point.longitude))
                if len(coordinates) == num_samples:
                    return coordinates  
    return coordinates 

gpx_data = {}

for file in os.listdir(gpx_folder):
    if file.endswith(".gpx"):
        raw_name = file.replace(".gpx", "").strip().lower()  
        formatted_name = raw_name.replace("_", "").replace(" ", "") 
        gpx_path = os.path.join(gpx_folder, file)
        gpx_data[formatted_name] = extract_gpx_coordinates(gpx_path)

gpx_df = pd.DataFrame(list(gpx_data.items()), columns=["PCS race entry", "race_coords"])

baseline = baseline.merge(gpx_df, on='PCS race entry', how='left')

KeyboardInterrupt: 

In [19]:
gpx_df

,PCS race entry,race_coords
0,2017amstelgoldrace,"[(50.87808, 5.71738), (50.87819, 5.71747), (50..."
1,2017brusselscyclingclassic,"[(50.83958, 4.39717), (50.83916, 4.39921), (50..."
2,2017debrabantsepijl-laflechebrabanconne,"[(50.87608, 4.70039), (50.87576, 4.70033), (50..."
3,2017dwarsdoorvlaanderen,"[(50.92555, 3.11799), (50.92559, 3.11739), (50..."
4,2017gent-wevelgeminflandersfields,"[(50.98226, 3.52379), (50.98278, 3.5238), (50...."
...,...,...
96,2023omloophetnieuwsblad,"[(50.99407, 3.7457), (50.99403, 3.74573), (50...."
97,2023paris-roubaix,"[(49.446, 2.89571), (49.44604, 2.89582), (49.4..."
98,2023rondevanvlaanderen-tourdesflandres,"[(51.1768, 3.31089), (51.17633, 3.31174), (51...."
99,2023scheldeprijs,"[(51.33058, 3.80118), (51.33076, 3.80116), (51..."


In [21]:
baseline

,Unnamed: 0,PCS race entry,Race Name,Year,Winner,Team,Winner Time,Avg Speed (km/h),2nd Place,3rd Place,...,date,profile_icon,profile_score,departure,arrival,average_temperature,start_time,race_datetime,race coords,race_coords
0,94,2017amstelgoldrace,Amstel Gold Race,2017,GILBERT Philippe,Quick-Step Floors,6:31:40,39.980,KWIATKOWSKI Michał,ALBASINI Michael,...,2017-04-16,p2,92,Maastricht,Berg en Terblijt,NaN,1900-01-01 11:26:00,2017-04-16 11:26:00,NaN,"[(50.87808, 5.71738), (50.87819, 5.71747), (50..."
1,95,2018amstelgoldrace,Amstel Gold Race,2018,VALGREN Michael,Astana Pro Team,6:35:07,39.940,KREUZIGER Roman,GASPAROTTO Enrico,...,2018-04-15,p2,103,Maastricht,Berg en Terblijt,NaN,1900-01-01 10:35:00,2018-04-15 10:35:00,NaN,"[(50.87808, 5.71738), (50.87819, 5.71747), (50..."
2,96,2019amstelgoldrace,Amstel Gold Race,2019,VAN DER POEL Mathieu,Corendon - Circus,6:28:18,41.056,CLARKE Simon,FUGLSANG Jakob,...,2019-04-21,p2,108,Maastricht,Berg en Terblijt,NaN,1900-01-01 12:35:00,2019-04-21 12:35:00,NaN,"[(50.86287, 5.70027), (50.86307, 5.70027), (50..."
3,97,2021amstelgoldrace,Amstel Gold Race,2021,VAN AERT Wout,Team Jumbo-Visma,5:03:29,43.218,PIDCOCK Thomas,SCHACHMANN Maximilian,...,2021-04-18,p2,81,Valkenburg,Berg en Terblijt,NaN,1900-01-01 12:11:00,2021-04-18 12:11:00,NaN,"[(50.86699, 5.78322), (50.86697, 5.78322), (50..."
4,98,2022amstelgoldrace,Amstel Gold Race,2022,KWIATKOWSKI Michał,INEOS Grenadiers,6:01:19,42.196,COSNEFROY Benoît,BENOOT Tiesj,...,2022-04-10,p2,112,Maastricht,Valkenburg,10.0,1900-01-01 10:26:00,2022-04-10 10:26:00,NaN,"[(50.86307, 5.70026), (50.86312, 5.70026), (50..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6,2019stradebianche,Strade Bianche,2019,ALAPHILIPPE Julian,Deceuninck - Quick Step,4:47:14,38.436,FUGLSANG Jakob,VAN AERT Wout,...,2019-03-09,p3,151,Siena,Siena,NaN,1900-01-01 10:45:00,2019-03-09 10:45:00,NaN,"[(43.31194, 11.31732), (43.31184, 11.31728), (..."
96,0,2020stradebianche,Strade Bianche,2020,VAN AERT Wout,Team Jumbo-Visma,4:58:56,36.930,FORMOLO Davide,SCHACHMANN Maximilian,...,2020-08-01,p3,159,Siena,Siena,NaN,1900-01-01 13:45:00,2020-08-01 13:45:00,NaN,"[(43.31122, 11.3171), (43.31121, 11.3171), (43..."
97,1,2021stradebianche,Strade Bianche,2021,VAN DER POEL Mathieu,Alpecin - Fenix,4:40:29,39.360,ALAPHILIPPE Julian,BERNAL Egan,...,2021-03-06,p3,159,Siena,Siena,NaN,1900-01-01 11:45:00,2021-03-06 11:45:00,NaN,"[(43.31122, 11.3171), (43.31121, 11.3171), (43..."
98,2,2022stradebianche,Strade Bianche,2022,POGAČAR Tadej,UAE Team Emirates,4:47:49,38.358,VALVERDE Alejandro,ASGREEN Kasper,...,2022-03-05,p3,167,Siena,Siena,NaN,1900-01-01 11:45:00,2022-03-05 11:45:00,NaN,"[(43.31086, 11.31682), (43.3108, 11.31679), (4..."


In [23]:
gpx_df["Race Name"] = gpx_df["PCS race entry"].map(race_name_mapping)

In [25]:
gpx_df['Year'] = gpx_df['PCS race entry'].str.extract(r'(\d{4})').astype(int)

In [27]:
gpx_df.loc[98, "Race Name"] = "Ronde van Vlaanderen"

In [29]:
gpx_df = gpx_df.sort_values(by=['Race Name', 'Year'], ascending=[True, True])

In [31]:
subset_baseline = baseline[['PCS race entry']]
subset_gpx = gpx_df[['PCS race entry']]

In [37]:
gpx_df[gpx_df.isnull().any(axis=1)]

,PCS race entry,race_coords,Race Name,Year


In [35]:
gpx_df.loc[49, "Race Name"] = "Liege - Bastogne - Liege"
gpx_df.loc[68, "Race Name"] = "Ronde van Vlaanderen"

In [30]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


In [32]:
import time
import os
from meteostat import Point, Hourly

In [34]:
def extract_points(gpx_file):
    with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)

    points = gpx.tracks[0].segments[0].points
    start = points[0]
    mid = points[len(points) // 2]
    end = points[-1]

    return [
        (start.latitude, start.longitude),
        (mid.latitude, mid.longitude),
        (end.latitude, end.longitude)
    ]

gpx_folder = r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\data\euro classics gpx"

gpx_files = [os.path.join(gpx_folder, file) for file in os.listdir(gpx_folder) if file.endswith('.gpx')]
total_points = {file: extract_points(file) for file in gpx_files}

In [36]:
total_points

{'C:\\Users\\mrmik\\OneDrive\\Desktop\\NSS\\Python\\Projects\\Cycling_Capstone\\data\\euro classics gpx\\2017 Amstel Gold Race.gpx': [(50.87808,
   5.71738),
  (50.7649, 5.95224),
  (50.85835, 5.80028)],
 'C:\\Users\\mrmik\\OneDrive\\Desktop\\NSS\\Python\\Projects\\Cycling_Capstone\\data\\euro classics gpx\\2017 Brussels Cycling Classic.gpx': [(50.83958,
   4.39717),
  (50.60256, 4.21849),
  (50.89434, 4.33171)],
 'C:\\Users\\mrmik\\OneDrive\\Desktop\\NSS\\Python\\Projects\\Cycling_Capstone\\data\\euro classics gpx\\2017 De Brabantse Pijl - La Fleche Brabanconne.gpx': [(50.87608,
   4.70039),
  (50.67499, 4.39356),
  (50.75711, 4.5247)],
 'C:\\Users\\mrmik\\OneDrive\\Desktop\\NSS\\Python\\Projects\\Cycling_Capstone\\data\\euro classics gpx\\2017 Dwars Door Vlaanderen .gpx': [(50.92555,
   3.11799),
  (50.84558, 3.64272),
  (50.87838, 3.43043)],
 'C:\\Users\\mrmik\\OneDrive\\Desktop\\NSS\\Python\\Projects\\Cycling_Capstone\\data\\euro classics gpx\\2017 Gent-Wevelgem in Flanders Fields.

In [38]:
import requests

def get_wind_data(lat, lon):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&hourly=windspeed_10m,winddirection_10m&timezone=auto"
    
    response = requests.get(url)
    data = response.json()

    wind_speed = data["hourly"]["windspeed_10m"][0] 
    wind_direction = data["hourly"]["winddirection_10m"][0]

    return wind_speed, wind_direction

In [42]:
results = []

for file, points in total_points.items():
    for i, (lat, lon) in enumerate(points):
        wind_speed, wind_dir = get_wind_data(lat, lon)
        results.append([file, i, lat, lon, wind_speed, wind_dir])

windspeed_df = pd.DataFrame(results, columns=["File", "Index", "Latitude", "Longitude", "Windspeed_kmh", "Wind_direction"])
print(windspeed_df)

windspeed_df.to_csv("wind_speed.csv", index=False)

                                                  File  Index  Latitude  \
0    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...      0  50.87808   
1    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...      1  50.76490   
2    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...      2  50.85835   
3    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...      0  50.83958   
4    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...      1  50.60256   
..                                                 ...    ...       ...   
340  C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...      1  51.40166   
341  C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...      2  51.25084   
342  C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...      0  43.31086   
343  C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...      1  43.16576   
344  C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...      2  43.31839   

     Longitude  Windspeed_kmh  Wind_direction  
0      5.71738           15.5             185  
1  

In [46]:
windspeed_df['File']

0      C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...
1      C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...
2      C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...
3      C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...
4      C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...
                             ...                        
340    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...
341    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...
342    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...
343    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...
344    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...
Name: File, Length: 345, dtype: object

In [245]:
#windspeed = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\wind_speed.xlsx")

In [48]:
windspeed_df['Cardinal_direction'] = ''

In [50]:
def get_cardinal_direction(degrees):
    directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW', 'N']
    index = round(degrees / 22.5) % 16
    return directions[index]

In [52]:
def get_cardinal_direction_df(df, wind_col='Wind_direction'):
    df['Cardinal_direction'] = df[wind_col].apply(get_cardinal_direction)

get_cardinal_direction_df(windspeed_df)

In [114]:
windspeed_df = windspeed_df.drop(columns=['Compass direction', 'Cardinel direction', 'Cardinal direction'])

KeyError: "['Compass direction', 'Cardinel direction', 'Cardinal direction'] not found in axis"

In [86]:
windspeed_df

,Race Name,Index,Latitude,Longitude,Windspeed_kmh,Wind_direction,Cardinal_direction,Year
0,2017 Amstel Gold Race,0,50.87808,5.71738,15.5,185,S,2017
1,2017 Amstel Gold Race,1,50.76490,5.95224,12.6,186,S,2017
2,2017 Amstel Gold Race,2,50.85835,5.80028,19.1,190,S,2017
3,2017 Brussels Cycling Classic,0,50.83958,4.39717,18.7,194,SSW,2017
4,2017 Brussels Cycling Classic,1,50.60256,4.21849,14.0,191,S,2017
...,...,...,...,...,...,...,...,...
340,2023 Scheldeprijs,1,51.40166,4.50678,15.1,158,SSE,2023
341,2023 Scheldeprijs,2,51.25084,4.49724,14.4,155,SSE,2023
342,2023 Strade Bianche,0,43.31086,11.31682,0.8,27,NNE,2023
343,2023 Strade Bianche,1,43.16576,11.46637,1.1,288,WNW,2023


In [84]:
windspeed_df.dtypes

Race Name              object
Index                   int64
Latitude              float64
Longitude             float64
Windspeed_kmh         float64
Wind_direction          int64
Cardinal_direction     object
Year                    int32
dtype: object

In [56]:
windspeed_df.at[0, 'File']

'C:\\Users\\mrmik\\OneDrive\\Desktop\\NSS\\Python\\Projects\\Cycling_Capstone\\data\\euro classics gpx\\2017 Amstel Gold Race.gpx'

In [58]:
windspeed_df['File'] = windspeed_df['File'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

In [66]:
windspeed_df['File'].unique()

array(['2017 Amstel Gold Race', '2017 Brussels Cycling Classic',
       '2017 De Brabantse Pijl - La Fleche Brabanconne',
       '2017 Dwars Door Vlaanderen ',
       '2017 Gent-Wevelgem in Flanders Fields', '2017 Il Lombardia',
       '2017 Kuurne-Bruxelles-Kuurne', '2017 La Fleche Wallonne',
       '2017 Liege - Bastogne - Liege', '2017 Milano - Sanremo',
       '2017 Milano - Torino', '2017 Omloop Het Nieuwsblad',
       '2017 Paris - Roubaix', '2017 Record Bank E3 Harelbeke',
       '2017 Ronde Van Vlaanderen', '2017 Scheldeprijs',
       '2017 Strade Bianche', '2018 Amstel Gold Race',
       '2018 Brussels Cycling Classic',
       '2018 De Brabantse Pijl - La Fleche Brabanconne',
       '2018 Dwars Door Vlaanderen ',
       '2018 Gent-Wevelgem in Flanders Fields', '2018 Il Lombardia',
       '2018 Kuurne-Bruxelles-Kuurne', '2018 La Fleche Wallonne',
       '2018 Liege - Bastogne - Liege', '2018 Milano - Sanremo',
       '2018 Milano - Torino', '2018 Omloop Het Nieuwsblad',
       

In [74]:
windspeed_df = windspeed_df.rename(columns={'File': 'Race Name'})

In [78]:
windspeed_df['Year'] = ''

In [80]:
windspeed_df['Year'] = windspeed_df['Race Name'].str.extract(r'(\d{4})').astype(int)

In [102]:
windspeed_df.to_excel('windspeed_df.xlsx')

In [92]:
windspeed_df = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\windspeed_df.xlsx")

In [94]:
windspeed_df

,Unnamed: 0,Race Name,Year,Index,Latitude,Longitude,Windspeed_kmh,Wind_direction,Cardinal_direction
0,0,Amstel Gold Race,2017,0,50.87808,5.71738,15.5,185,S
1,1,Amstel Gold Race,2017,1,50.76490,5.95224,12.6,186,S
2,2,Amstel Gold Race,2017,2,50.85835,5.80028,19.1,190,S
3,6,De Brabantse Pijl,2017,0,50.87608,4.70039,15.1,189,S
4,7,De Brabantse Pijl,2017,1,50.67499,4.39356,16.9,187,S
...,...,...,...,...,...,...,...,...,...
298,340,Scheldeprijs,2023,1,51.40166,4.50678,15.1,158,SSE
299,341,Scheldeprijs,2023,2,51.25084,4.49724,14.4,155,SSE
300,342,Strade Bianche,2023,0,43.31086,11.31682,0.8,27,NNE
301,343,Strade Bianche,2023,1,43.16576,11.46637,1.1,288,WNW


In [98]:
windspeed_df = windspeed_df.sort_values(by=['Race Name', 'Year'], ascending=[True, True])

In [134]:
pps = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\rider_points.xlsx")

In [60]:
baseline

,Unnamed: 0,PCS race entry,Race Name,Year,Winner,GC speciality points,time_trial speciality points,sprint speciality points,climber speciality points,hills speciality points,...,Won How,date,profile_icon,profile_score,departure,arrival,average_temperature,start_time,race_datetime,race_coords
0,0,2017amstelgoldrace,Amstel Gold Race,2017,GILBERT Philippe,4215.0,1426.0,1094.0,3215.0,5140.0,...,Sprint à deux,2017-04-16,"Hills, flat finish",92.0,Maastricht,Berg en Terblijt,NaN,1900-01-01 11:26:00,2017-04-16 11:26:00,"[(50.87808, 5.71738), (50.87819, 5.71747), (50..."
1,1,2018amstelgoldrace,Amstel Gold Race,2018,VALGREN Michael,1188.0,453.0,84.0,955.0,1858.0,...,Sprint à deux,2018-04-15,"Hills, flat finish",103.0,Maastricht,Berg en Terblijt,NaN,1900-01-01 10:35:00,2018-04-15 10:35:00,"[(50.87808, 5.71738), (50.87819, 5.71747), (50..."
2,2,2019amstelgoldrace,Amstel Gold Race,2019,VAN DER POEL Mathieu,1119.0,968.0,1286.0,1042.0,3090.0,...,Sprint of small group,2019-04-21,"Hills, flat finish",108.0,Maastricht,Berg en Terblijt,NaN,1900-01-01 12:35:00,2019-04-21 12:35:00,"[(50.86287, 5.70027), (50.86307, 5.70027), (50..."
3,3,2021amstelgoldrace,Amstel Gold Race,2021,VAN AERT Wout,1171.0,2944.0,1886.0,2044.0,4221.0,...,Sprint of small group,2021-04-18,"Hills, flat finish",81.0,Valkenburg,Berg en Terblijt,NaN,1900-01-01 12:11:00,2021-04-18 12:11:00,"[(50.86699, 5.78322), (50.86697, 5.78322), (50..."
4,4,2022amstelgoldrace,Amstel Gold Race,2022,KWIATKOWSKI Michał,3670.0,3561.0,755.0,3304.0,3579.0,...,Sprint à deux,2022-04-10,"Hills, flat finish",112.0,Maastricht,Valkenburg,10.0,1900-01-01 10:26:00,2022-04-10 10:26:00,"[(50.86307, 5.70026), (50.86312, 5.70026), (50..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3,2019kuurne-brussel-kuurne,Kuurne-Brussel-Kuurne,2019,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
96,3,2020kuurne-brussel-kuurne,Kuurne-Brussel-Kuurne,2020,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
97,3,2021kuurne-brussel-kuurne,Kuurne-Brussel-Kuurne,2021,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
98,3,2022kuurne-brussel-kuurne,Kuurne-Brussel-Kuurne,2022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN


In [146]:
pps = pps.rename(columns={'Rider': 'Winner'})

In [150]:
baseline = pd.merge(baseline, pps, on='Winner', how='inner')

In [158]:
for index, column_name in enumerate(baseline.columns):
    print(f"Column index: {index}, Column name: {column_name}")

Column index: 0, Column name: Unnamed: 0
Column index: 1, Column name: PCS race entry
Column index: 2, Column name: Race Name
Column index: 3, Column name: Year
Column index: 4, Column name: Winner
Column index: 5, Column name: gc
Column index: 6, Column name: time_trial
Column index: 7, Column name: sprint
Column index: 8, Column name: climber
Column index: 9, Column name: hills
Column index: 10, Column name: Team
Column index: 11, Column name: Winner Time
Column index: 12, Column name: Avg Speed (km/h)
Column index: 13, Column name: 2nd Place
Column index: 14, Column name: 3rd Place
Column index: 15, Column name: 4th Place
Column index: 16, Column name: 5th Place
Column index: 17, Column name: Time Gaps
Column index: 18, Column name: Won How
Column index: 19, Column name: date
Column index: 20, Column name: profile_icon
Column index: 21, Column name: profile_score
Column index: 22, Column name: departure
Column index: 23, Column name: arrival
Column index: 24, Column name: average_te

In [156]:
baseline = baseline.iloc[:, [0,1,2,3,4,25,26,27,28,29, 5,6,7,8,9, 10, 11,12,13,14,15,16,17,18,19,20,21,22,23]]

In [162]:
baseline = baseline.rename(columns={'gc': 'GC speciality points', 'time_trial': 'time_trial speciality points', 'sprint': 'sprint speciality points', 'climber': 'climber speciality points', 'hills': 'hills speciality points'})

In [168]:
baseline.at[0, 'race_coords']

[(50.87808, 5.71738),
 (50.87819, 5.71747),
 (50.88, 5.71888),
 (50.8801, 5.71896),
 (50.8802, 5.71904),
 (50.88235, 5.72067),
 (50.88285, 5.72115),
 (50.88292, 5.72124),
 (50.88329, 5.72167),
 (50.88382, 5.72249)]

In [ ]:
windspeed_df.at[0, 'File']

In [172]:
baseline = baseline.drop(columns={'Unnamed: 0', 'race coords'})

In [174]:
baseline

,PCS race entry,Race Name,Year,Winner,GC speciality points,time_trial speciality points,sprint speciality points,climber speciality points,hills speciality points,Team,...,Won How,date,profile_icon,profile_score,departure,arrival,average_temperature,start_time,race_datetime,race_coords
0,2017amstelgoldrace,Amstel Gold Race,2017,GILBERT Philippe,4215,1426,1094,3215,5140,Quick-Step Floors,...,Sprint à deux,2017-04-16,p2,92,Maastricht,Berg en Terblijt,NaN,1900-01-01 11:26:00,2017-04-16 11:26:00,"[(50.87808, 5.71738), (50.87819, 5.71747), (50..."
1,2018amstelgoldrace,Amstel Gold Race,2018,VALGREN Michael,1188,453,84,955,1858,Astana Pro Team,...,Sprint à deux,2018-04-15,p2,103,Maastricht,Berg en Terblijt,NaN,1900-01-01 10:35:00,2018-04-15 10:35:00,"[(50.87808, 5.71738), (50.87819, 5.71747), (50..."
2,2019amstelgoldrace,Amstel Gold Race,2019,VAN DER POEL Mathieu,1119,968,1286,1042,3090,Corendon - Circus,...,Sprint of small group,2019-04-21,p2,108,Maastricht,Berg en Terblijt,NaN,1900-01-01 12:35:00,2019-04-21 12:35:00,"[(50.86287, 5.70027), (50.86307, 5.70027), (50..."
3,2021amstelgoldrace,Amstel Gold Race,2021,VAN AERT Wout,1171,2944,1886,2044,4221,Team Jumbo-Visma,...,Sprint of small group,2021-04-18,p2,81,Valkenburg,Berg en Terblijt,NaN,1900-01-01 12:11:00,2021-04-18 12:11:00,"[(50.86699, 5.78322), (50.86697, 5.78322), (50..."
4,2022amstelgoldrace,Amstel Gold Race,2022,KWIATKOWSKI Michał,3670,3561,755,3304,3579,INEOS Grenadiers,...,Sprint à deux,2022-04-10,p2,112,Maastricht,Valkenburg,10.0,1900-01-01 10:26:00,2022-04-10 10:26:00,"[(50.86307, 5.70026), (50.86312, 5.70026), (50..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2019stradebianche,Strade Bianche,2019,ALAPHILIPPE Julian,2725,1427,774,5233,4583,Deceuninck - Quick Step,...,Sprint à deux,2019-03-09,p3,151,Siena,Siena,NaN,1900-01-01 10:45:00,2019-03-09 10:45:00,"[(43.31194, 11.31732), (43.31184, 11.31728), (..."
96,2020stradebianche,Strade Bianche,2020,VAN AERT Wout,1171,2944,1886,2044,4221,Team Jumbo-Visma,...,12.5 km solo,2020-08-01,p3,159,Siena,Siena,NaN,1900-01-01 13:45:00,2020-08-01 13:45:00,"[(43.31122, 11.3171), (43.31121, 11.3171), (43..."
97,2021stradebianche,Strade Bianche,2021,VAN DER POEL Mathieu,1119,968,1286,1042,3090,Alpecin - Fenix,...,0.5 km solo,2021-03-06,p3,159,Siena,Siena,NaN,1900-01-01 11:45:00,2021-03-06 11:45:00,"[(43.31122, 11.3171), (43.31121, 11.3171), (43..."
98,2022stradebianche,Strade Bianche,2022,POGAČAR Tadej,6344,2623,227,7973,3330,UAE Team Emirates,...,49.2 km solo,2022-03-05,p3,167,Siena,Siena,NaN,1900-01-01 11:45:00,2022-03-05 11:45:00,"[(43.31086, 11.31682), (43.3108, 11.31679), (4..."


In [176]:
baseline.to_excel('baseline_data.xlsx')

In [180]:
profile_icons = {
    'p1': 'Flat',
    'p2': 'Hills, flat finish',
    'p3': 'Hills, uphill finish',
    'p4': 'Mountains, flat finish',
    'p5': 'Mountains, uphill finish'
}

In [182]:
baseline['profile_icon'] = baseline['profile_icon'].map(profile_icons)

In [194]:
baseline.to_excel('baseline_data.xlsx')

In [188]:
baseline['profile_icon'].value_counts()

profile_icon
Hills, flat finish          52
Hills, uphill finish        27
Flat                        20
Mountains, uphill finish     1
Name: count, dtype: int64

In [190]:
#consider whether to do this for races, would need a new datasheet and relevant key to match URL

from procyclingstats import RaceClimbs
race_climbs = RaceClimbs('race/ronde-van-vlaanderen/2023/route/climbs')
race_climbs.climbs()

[{'climb_name': 'Oude Kwaremont',
  'climb_url': 'location/oude-kwaremont',
  'length': 2.6,
  'steepness': 3.6,
  'top': 104,
  'km_before_finnish': 55},
 {'climb_name': 'Oude Kwaremont',
  'climb_url': 'location/oude-kwaremont',
  'length': 2.5,
  'steepness': 3.5,
  'top': 101,
  'km_before_finnish': 137},
 {'climb_name': 'Oude Kwaremont',
  'climb_url': 'location/oude-kwaremont',
  'length': 2.5,
  'steepness': 3.5,
  'top': 101,
  'km_before_finnish': 17},
 {'climb_name': 'Kruisberg/Hotond',
  'climb_url': 'location/kruisberg-hotond',
  'length': 2.5,
  'steepness': 4.3,
  'top': 159,
  'km_before_finnish': 26},
 {'climb_name': 'Marlboroughstraat',
  'climb_url': 'location/marlboroughstraat',
  'length': 1.8,
  'steepness': 3.1,
  'top': 98,
  'km_before_finnish': 98},
 {'climb_name': 'Eikenberg',
  'climb_url': 'location/eikenberg',
  'length': 1.2,
  'steepness': 5.0,
  'top': 82,
  'km_before_finnish': 118},
 {'climb_name': 'Kanarieberg',
  'climb_url': 'location/kanarieberg',


In [198]:
#consider whether you want to try and calculate time differences between the places 